In [ ]:
import os
import sys

src_dir = os.path.abspath("../src")
if src_dir not in sys.path:
    sys.path.append(src_dir)

In [16]:
# import the needed packages
import matplotlib.pyplot as plt
import numpy as np
import tools
import importlib
importlib.reload(tools)

import solver
import importlib
importlib.reload(solver)

from evolution_chained2_kicked import evolution_chained2_kicked
from tools import mv

In [17]:
# create bath

# chain size
ns = 30
# on-site energies
e = [1.0]*ns
# hoppings between sites
h = [0.05]*(ns-1)

In [18]:
# discretize the bath: apply the lightcone method
bth = solver.bath.from_chain(chain = (e, h), rel_tol = 10**(-9), ring_size = 6, t_max = 100, visualize = True)
bth.n_rel

10

In [19]:
num_modes = 30
max_num_quanta = 4
m = tools.spin_boson_model(num_modes, max_num_quanta)

wq = 1.0
Hs = wq * m.s_p @ m.s_m
g = 0.1
V = g * m.s_m
V_dag = g * m.s_p

def f(ti):
    return(0.1*np.cos((ti + 0.5)*bth.dt))

psi = np.zeros(m.dimension, dtype = complex)
psi_mid = np.zeros(m.dimension, dtype = complex)
psi_mid_next = np.zeros(m.dimension, dtype = complex)
psi_buff = np.zeros(m.dimension, dtype = complex)

psi_ini = np.zeros(m.dimension, dtype = complex)
psi_ini[0] = 1
psi_begin = np.copy(psi_ini)

Ht = None

sz_av = []

def begin_step(ti, psi):
    m_in = bth.m_in(ti)
    m_out = bth.m_out(ti)
    Hint = V_dag @ sum(bth.couplings_cdia[:, ti] * m.a[:])
    Hint = Hint + Hint.conj().transpose()
    Hw = m.zero_op.copy()
    w = bth.w_cdia(ti)
    if not w is None:
        for p in range(m_out, m_in):
            for q in range(m_out, m_in):
                Hw += 1j * m.a_dag[q] @ m.a[p] * w[q - m_out, p - m_out].conj()
    global Ht
    Ht = Hs + m.s_x * f(ti) + Hint + Hw
    
def apply_H(ti, psi_in, psi_out):
    mv(Ht, psi_in, psi_out)

def eval_o(ti, psi):
    sz_av.append(np.vdot(psi, m.s_p @ m.s_m @ psi))

first_in_chain = 1
evolution_chained2_kicked(0, bth.nt-2, bth.dt, begin_step, apply_H, eval_o, psi_begin, psi, psi_mid, psi_mid_next, first_in_chain)
psi_cd_full = np.copy(psi) / np.sqrt(np.vdot(psi, psi))

In [20]:
Nq = sum([m.a_dag[i] @ m.a[i] for i in range(20)])

In [21]:
np.vdot(psi_cd_full, Nq @ psi_cd_full)

(2.254662594822959+0j)

In [11]:
np.vdot(psi_cd_full, Nq @ psi_cd_full)

(2.254658244612255+3.469446951953614e-18j)

In [22]:
from evolution_chained2_kicked import evolution_chained2_kicked
from tools import mv

num_modes = 30
max_num_quanta = 4
m = tools.spin_boson_model(num_modes, max_num_quanta)

wq = 1.0
Hs = wq * m.s_p @ m.s_m
g = 0.1
V = g * m.s_m
V_dag = g * m.s_p

def f(ti):
    return(0.1*np.cos((ti + 0.5)*bth.dt))

psi = np.zeros(m.dimension, dtype = complex)
psi_mid = np.zeros(m.dimension, dtype = complex)
psi_mid_next = np.zeros(m.dimension, dtype = complex)
psi_buff = np.zeros(m.dimension, dtype = complex)

psi_ini = np.zeros(m.dimension, dtype = complex)
psi_ini[0] = 1
psi_begin = np.copy(psi_ini)

Ht = None

sz_av_1 = []

def begin_step(ti, psi):
    m_in = bth.m_in(ti)
    m_out = bth.m_out(ti)
    Hint = V_dag @ sum(bth.couplings_cdia[m_out : m_in, ti] * m.a[m_out : m_in])
    Hint = Hint + Hint.conj().transpose()
    Hw = m.zero_op.copy()
    w = bth.w_cdia(ti)
    if not w is None:
        for p in range(m_out, m_in):
            for q in range(m_out, m_in):
                Hw += 1j * m.a_dag[q] @ m.a[p] * w[q - m_out, p - m_out].conj()
    global Ht
    Ht = Hs + m.s_x * f(ti) + Hint + Hw
    
def apply_H(ti, psi_in, psi_out):
    mv(Ht, psi_in, psi_out)

def eval_o(ti, psi):
    sz_av_1.append(np.vdot(psi, m.s_p @ m.s_m @ psi))
    #pass

first_in_chain = 1
evolution_chained2_kicked(0, bth.nt-2, bth.dt, begin_step, apply_H, eval_o, psi_begin, psi, psi_mid, psi_mid_next, first_in_chain)
psi_cd = np.copy(psi) / np.sqrt(np.vdot(psi, psi))

In [23]:
np.vdot(psi_cd, Nq @ psi_cd)

(2.2547150397685436+8.673617379884035e-18j)

In [24]:
f = 1 - np.abs(np.vdot(psi_cd, psi_cd_full))
f

3.8397330015538245e-07